In [8]:
import pandas as pd
import numpy as np
import util

enrollment_path = './data/nj/enrollment/'

years = [
    2017
    , 2018
    , 2019
]

enrollment_files = [
    'nj_enrollment_2017_cleaned.csv'
    , 'nj_enrollment_2018_cleaned.csv'
    , 'nj_enrollment_2019_cleaned.csv'
]

# import main enrollment files
df = pd.read_csv(enrollment_path + enrollment_files[0])
df['year'] = df.apply(lambda x: years[0], axis=1)

for i in range(1, len(enrollment_files)):
    df2 = pd.read_csv(enrollment_path + enrollment_files[i])
    df2['year'] = df2.apply(lambda x: years[i], axis=1)
    df = df.append(df2, ignore_index = True, sort=True)

In [9]:
# rename TOTAL to All Grades
df['grade'] = df['grade'].str.upper()
df['grade'] = [x if x != 'TOTAL' else 'All Grades' for x in df['grade']]

0                 5
1                 6
2                 7
3                 8
4                UG
5        All Grades
6                PK
7                KG
8                 1
9                 2
10                3
11                4
26                9
27               10
28               11
29               12
26473            PF
26474            KF
26669            PH
26771            KH
Name: grade, dtype: object


In [11]:
# collapse genders into a single subgroup column
gender_subgroups = [
    'AM'
    , 'AS'
    , 'BL'
    , 'HI'
    , 'MU'
    , 'PI'
    , 'WH'
]

# combine genders for every subgroup
for subgroup in gender_subgroups:
    df[subgroup] = df.apply(lambda x: x[subgroup + '_M'] + x[subgroup + '_F'], axis=1)

   AM_F  AM_M  AS_F  AS_M  All Groups  BL_F  BL_M  COUNTY_ID COUNTY_NAME  \
0   0.0   0.0   0.0   1.0        86.0  11.0   3.0          1    ATLANTIC   
1   0.0   0.0   0.0   1.0        84.0   5.0  10.0          1    ATLANTIC   
2   0.0   0.0   0.0   0.0       105.0   7.0   8.0          1    ATLANTIC   
3   0.0   0.0   0.0   1.0        75.0   6.0   3.0          1    ATLANTIC   
4   0.0   0.0   0.0   0.0         5.0   1.0   3.0          1    ATLANTIC   

   FREE_LUNCH  ...          school  school_id  year   AM   AS    BL    HI  \
0         NaN  ...  Emma C Attales         50  2017  0.0  1.0  14.0  11.0   
1         NaN  ...  Emma C Attales         50  2017  0.0  1.0  15.0   4.0   
2         NaN  ...  Emma C Attales         50  2017  0.0  0.0  15.0  11.0   
3         NaN  ...  Emma C Attales         50  2017  0.0  1.0   9.0   7.0   
4         NaN  ...  Emma C Attales         50  2017  0.0  0.0   4.0   0.0   

     MU   PI    WH  
0   9.0  0.0  51.0  
1   4.0  0.0  60.0  
2   9.0  0.0  70.

In [12]:
# reshape data
subgroups = [
    'AM'
    , 'AS'
    , 'BL'
    , 'HI'
    , 'MU'
    , 'PI'
    , 'WH'
    , 'FREE_LUNCH'
    , 'LEP'
    , 'MIGRANT'
    , 'All Groups'
]
df = pd.melt(df, id_vars=['year', 'district_id', 'district', 'school_id', 'school', 'grade']
        , value_vars=subgroups
        , var_name='group_state'
        , value_name='num')

In [14]:
# drop district results
df = df[df['school_id'] != 999]

In [15]:
# export
df.to_csv('./data/finalized/nj_enrollment.csv', index=False)